In [1]:
"""Bond Breaking"""

__authors__ = "Victor H. Chavez", "Lyudmila Slipchenko"
__credits__ = ["Victor H. Chavez", "Lyudmila Slipchenko"]
__email__   = ["gonza445@purdue.edu", "lslipchenko@purdue.edu"]

__copyright__ = "(c) 2008-2019, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2019-11-18"

---
## Lab 2. Bond-breaking in $H_2$

In this lab, you will:
* Investigate the bond-breaking reaction in $H_2$ molecule.
* Compare the performance of restricted and unrestricted Hartree-Fock, and Density Functional Theory for bond breaking.
* Benchmark these results with respect to the Full Configuration Interaction (FCI) values obtained using the coupled cluster with single and double excitations (CCSD) calculations, which give the exact answer for the two-electron system. 
* Calculate the correlation energy.
* Distinguish dynamic and static contributions to the correlation energy.

Authors: Lyudmila Slipchenko (lslipchenko@purdue.edu; ORCID: 0000-0002-0445-2990) and Victor H. Chavez (gonza445@purdue.edu; ORCID: 0000-0003-3765-2961).


***

In [3]:
#Import modules

import psi4

import numpy as np
import os
import matplotlib.pyplot as plt


***
To perform a basic calculation we use the ```psi4.energy``` function. The function needs to know what method and basis set to use, and what molecule you are interested in (if you have defined more than one geometry inside your Jupyter Notebook). Let say that we want to get the HF energy of the Helium atom. We would need to do the following:


In [4]:
#Define Helium Geometry
#The first line referst to the charge and spin multiplicity. 

he_geo = psi4.geometry("""
0 1
He 0.0 0.0 0.0
""")

#Request the HF calculation using the correlation consistent basis set cc-pvdz. 
e = psi4.energy("HF/cc-pvdz", molecule=he_geo)


#Print the energy. The units are given in atomic units or hartrees. 
print(f"The HF energy of He is {e}")

#We made us of *f-strings* which allow us to combine strings and numbers in a print statement.

The HF energy of He is -2.8551883987268125


If you were to try the Helium example on a Hydrogen atom as it is, you would find that Psi4 will throw an error. This is because when running a calculation, Psi4 defaults to a *Restricted Hartree-Fock* or *RHF*, (i.e. a system with an even number of electrons where all electrons are paired). This means that electrons of opposite spin occupy (or are "restricted") to the same spatial orbital. 

In cases like Hydrogen, where the numbers of alpha and beta spin electrons are different, we lift this restriction allowing both electrons to have different spatial orbitals. 

<br>

<img src="./restricted.png">

<br>

We need to tell Psi4 that we want an UHF calculation. This is done by setting the global option "reference" as "UHF". In the cell below, type: ```psi4.set_options({"reference" : "UHF"})```. You may need to switch between UHF and RHF many times throughout the lab. 

***

We want to produce a binding energy curve for the $H_2$ molecule using different levels of theory. The binding energy is given by:

 $$E_{bind} = E(H_2) - 2E(H)  \tag{1} $$

For a molecule with one degree of freedom, just like the $H_2$ molecule, the potential energy surface is just a 1D curve. Notice that the second term on the right hand side of the equation is just constant that is equal to two times the energy of the Hydrogen atom. Your first task is to obtain this value for each method:

### Part 1


#### **1.** Calculate and store the energy of a single H atom with the methods: HF, PBE, B3LYP and CCSD. Use 6-31G** basis for all the calculation in this lab. Change the reference and multiplicity of the atom accordingly.  


<div class="alert alert-info">
    
Hint: Notice that the first argument of `psi4.energy` is a string. You could quickly go through the calculations by creating a list with the different methods and then use them in a for loop to to run each of them. Consider that the string also contains the basis set. In order to overcome this predicament, remember that strings can be concatenated by using the `+` operator (e.g.  "HF"+"/cc-pvdz"). 
    
</div>

#### **2.** Explain the origin of errors in each method and why HF and CCSD energies are the same for the H atom. 


In [2]:
### RESPONSE

---

Let us now concentrate on the first term of equation 1. We require to run a series of calculations for each method at different H-H separations in Angstroms (e.g. 0.3, 0.4, 0.5, ... ,4.9, 5.0).  
<div class="alert alert-info">
Hint : Given that the argument of a psi4.geometry is a string, we can take advantage of that by looking at the following example:
</div>

In [4]:
#Define string with psi4.geometry syntax. 
#Identify what you want to change and use a particular label that you know that won't get repeated. 
molecule = """ 
**atom1** 0.0 0.0 0.0
"""

#Create a list with the things that you want to go through. 
atoms = [ "H", "He", "Li" ]

#Cycle through them. 
for atom in atoms:
    print(molecule.replace("**atom1**", atom))

 
H 0.0 0.0 0.0

 
He 0.0 0.0 0.0

 
Li 0.0 0.0 0.0



#### **3.** Using the previous example and the following distances, write a snippet that will calculate the energy at each separation for a **RHF** calculation. You will need to change the reference to "RHF" (Psi4 still thinks you want to run UHF calculations).  
Make sure you store the wavefunction object for each separation since it will be used later in the lab: ```energy, wfn = psi4.energy("method/basis", return_wfn=True)```

In [5]:
#We use more points closer to where we would expect to have the ground state geometry to create a nice and smooth function. 
distances = np.zeros(20)
distances[0:16] = np.linspace(0.3, 2.5, 16)
distances[16:] = np.linspace(2.7, 5.0, 4)

Here we are using the `numpy` library first to create an empty array filled with zeros using `np.zeros`, where the argument specifies the size of the array. The other function `np.linspace` creates a sequence of evenly spaced values within an interval. This means, we generated a linear space with 16 points from 0.3 to 2.5 and one with 4 points from 2.7 to 5.0. 

In [3]:
#RESPONSE:


---
#### **4.** Calculate the energies at the same distances at the **UHF** level. You can recycle the code that you just wrote (just remember to change the name of your variables).  We will need extra information that can ony be found in the output. 

In order to save the output to a file we require the additional option: ```psi4.core.set_output_file("filename.txt", True)```.

<div class="alert alert-info">
Hint: In order to obtain the correct UHF energies, we need to set the extra following options:
<\div>


In [7]:
psi4.set_options({'reference' : 'UHF', 
                  'guess_mix' : True, 
                  'guess' : "gwh"})

In [5]:
#RESPONSE

#### **5.** Store the values for $S^2$. This information is found in each outputfile close to the end of your calculation (look for Spin Contamination Metric). 

You can go through each of the files and copy the value, but you can also think about how can you let python automatize this process. Think carefully about the steps required for this. Given a path you would need to import the file ( `f = open(path, 'r')` ) and proceed to extract the lines ( `f.read().splitlines()` ). With those lines available, you may concentrate on determining whether or not each line contains the `S^2` string. 

If you require a more thorough review of parsing files. You can look at [this tutorial](https://education.molssi.org/python_scripting_cms/02-file_parsing/index.html) to learn more about file parsing. 

In [6]:
#Response


#### **6.** Make a table or plot of $S^2$ values from the UHF calculations. Explain why $S^2$ deteriorates when the H-H bond is stretched.

In [7]:
#Response

---
#### 7. Calculate the same potential energy surface at the DFT level. Use the PBE functional and a restricted wavefunction.  

In [8]:
#RESPONSE


#### **8.** Calculate the same potential energy surface at the FCI level. 
For a two-electron system, the FCI results may be obtained by using the CCSD method. This is true because CCSD includes determinants that are singly and doubly excited. For a two electron system that includes all electrons available, thus CCSD includes all possible excitations in the system. 

#### **9.** You will need to save the output file generated by Psi4 again. From the output file, record total CCSD ampltitudes: CCSD $T_1^2$ and $T_2^2$, and the value of the largest $T_2$ amplitude for the ground state geometry and for a split geometry. 

Consider T as a sum of operators that that act on a reference determinant. In CCSD $T = T_1 + T_2$ where $T_1$ refers with single excitations and $T_2$ with double excitations. 
The values of amplitudes show a relative weight of singly and doubly excited determinants in the wavefunction. If $T_1$ and/or $T_2$ are large (generally speaking, if a particular |$T_2| > 0.1$), the wavefunction is considered to be multi-configurational, i.e., containing several important Slater determinants. In other words, this is a region where non-dynamic (static) correlation is significant. Several small $T_1$ and $T_2$ amplitudes tell about (almost always present) dynamic correlation.

In each output you should look at the values of *Largest {TIA, Tia, TIjAb} Amplitudes*, where the $T$ refers to the previously mentioned operator, and the following indices refer to the orbitals used according to the notation:


|       | Occupied Molecular Orbitals | Virtual Molecular Orbitals |   |   |
|-------|-----------------------------|----------------------------|---|---|
| Alpha | i,j                         | a, b                       |   |   |
| Beta  | I, J                        | A, B                       |   |   |


In [10]:
#RESPONSE

---

#### **10.** Plot on the same graph the RHF, DFT and FCI binding energies in $H_2$ versus the separation distance. Plot in kcal/mol energy units (1 Hartree = 627.5 kcal/mol)

In [12]:
#Response

#### **11.** Using your results, compare CCSD, UHF and RHF dissociation energies. 

In [ ]:
#RESPONSE

---
#### **12.** Comment on the behaviour of RHF with respect to FCI at short (around 0.7 Angstroms) and long distances. For more information, you can read paragraph 3.8.7 from Reference 1 (found below) for a discussion of RHF and UHF solutions. 

#RESPONSE

---
#### **13.** Plot the first two $H_2$ molecular orbitals from your RHF and UHF calculations at equilibrium , 0.7 and 5.0 Angstroms. Remember to use the appropriate global settings. Comment on qualitative changes in the shape of the orbitals.

##### You may use the function `generate_orbitals` from the orbital_helper file in the same directory to plot both HOMO and HOMO for the $H_2$ molecule. The syntax is the following:
```
from orbital_helper import generate_orbitals
x, alpha_orbitals, beta_orbitals = generate_orbitals(wfn, [1,2,3])
#Where the arguments are the wavefunction object and the integer values of the orbitals.
#The function returns a numpy array with the domain, and a set of lists with alpha and beta orbitals. 
```

##### If you have the package `moly` installed. You may visualize the orbitals in 3D with the following:
```
import moly
fig = moly.Figure(figsize=(300,500))
fig.add_orbital("Name", wfn, orbital_number, iso, colorscale="portland_r")
fig.show()
```


In [14]:
from orbital_helper import generate_orbitals

In [13]:
#RESPONSE

---
#### **14.** Difference between FCI and HF energies is the correlation energy. What is the nature of the correlation energy (dynamic vs non-dynamic) in $H_2$ at equilibrium and long distances? At what distance does the non-dynamic correlation become important?

#RESPONSE

---
#### **15.** Comment on the behaviour of DFT at equilibrium and long distances. What is the reason of DFT failure for bond-breaking?

#RESPONSE

---
#### **Bonus.** From the previously computed energy of a Hydrogen atom with the hybrid B3LYP functional. Compare the energy of the atom computed with HF, B3LYP and the exact energy. Do you see any discrepancy with B3LYP? If so, what is/are the reasons for such discrepancies?

In [14]:
#RESPONSE

***
## Part 2

Your friend, who is an experimental chemist, seeks your help knowing that you have expertise in running quantum chemistry simulations. Their research group has measured the singlet-triplet gap of ozone recently. They want to see if computational simulations can support their measurement. How will you measure the singlet-triplet gap in ozone?

Use the ideas from the previous part of this lab and the follwoing hints:
    
   **1.** Assume that the singlet and triplet ozone molecules have the same geometry. 
   
   **2.** You will have to optimize the geometry of ozone to start with. Psi4 can let you import geometries from PubChem. The sytax is: `h2o_geometry = psi4.geometry("pubchem:water")`. You may use the common name or its molecular formula. Alternatively, you can use a database such as [CCCBDB](https://cccbdb.nist.gov/). 
   
   **3.** Use RHF/6-31G* for simulating the singlet ozone molecule. Use UHF/6-31G* for simulating the triplet ozone molecule. Use the energy difference to compute the gap.  
   
   **4.** Write the electronic energies corresponding to singlet and triplet ozone molecules. the singlet-triplet gap in eV, and the $<S^2>$ value for triplet ozone. Information about spin contamination is given by $<S^2>$ and can be found close to the end of your calculation (look for Spin Contamination Metric). 
    
   
    
   
    

In [16]:
#Response

---
Now, compute the singlet-triplet gap between the $^1\Delta_g$ and $^3\Sigma_g$ states of oxygen molecule and report it in eV. Compare the singlet-triplet gap you computed in this lab with the ones availiable in CCBDB. Is it an exact match (http://cccbdb.nist.gov/stgap1.asp)?


<img src="./ozone.png">

##### Compare the expected $<S^2>$ with observed $<S^2>$ and respond: Of all the four cases you have computed so far, which one suffers the most spin contamination?

In [17]:
#RESPONSE

---
Bonus. Compute the singlet-triplet gap between $^1\Sigma_g ^+$ and $^3\Sigma_g ^-$ states of oxygen atom.  
        
   
<div class="alert alert-info">

Hint: Start with $^1 \Delta_g$ geometry. Use the maximum overlap method (MOM) to force the highest beta electron to occupy the second $\pi^*$ ortibal: ```psi4.set_options({"MOM_START":1})``` 
       
</div>

In [18]:
#Response


# ---
## Further Reading:

#### General:
1. Szabo, A., & Ostlund, N. S. (2012). Modern quantum chemistry: introduction to advanced electronic structure theory. Courier Corporation.  
2. Cramer, Christopher J. Essentials of computational chemistry: theories and models. John Wiley & Sons, 2013.  
3. Krylov. A. Theory and Practice of Molecular Electronic Structure: [link](http://iopenshell.usc.edu/chem545/lectures2016/chem545_2016.pdf)
4. Sherrill. D. Non-Dynamical (Static) Electron Correlation: Bond Breaking in Quantum Chemistry [link](https://youtu.be/coGVX7HCCQE)


#### Bond stretching:
1. Dutta, Antara, and C. David Sherrill. "Full configuration interaction potential energy curves for breaking bonds to hydrogen: An assessment of single-reference correlation methods." The Journal of chemical physics 118.4 (2003): 1610-1619.

#### Singlet-triplet gaps:
1. Slipchenko, Lyudmila V., and Anna I. Krylov. "Singlet-triplet gaps in diradicals by the spin-flip approach: A benchmark study." The Journal of chemical physics 117.10 (2002): 4694-4708.
